In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from PIL import Image


In [ ]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

In [ ]:
df = pd.read_csv("../input/train.csv")

In [ ]:
df["Target"]=df["Target"].apply(lambda x:x.split(" "))


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

image[:,:,0] = imread("../input/train/")

In [ ]:
df.head(5)

In [ ]:
df.fillna(0, inplace=True)


In [ ]:
df.loc[df.Id=="eef76f56-bb99-11e8-b2b9-ac1f6b6435d0"]

In [ ]:
#to be used in various classes,created for singularity
class Mparam:
    def __init__(self,basepath,imgrows=512,imgcol=512,rowscalefac=4,columnscalefac=4,epoch=1,batchsize=100,n_channels=1,num_classes=28,shuffle=False):
        self.imgrows = imgrows
        self.imgcol=imgcol
        self.basepath = basepath
        self.rowscalefac = rowscalefac
        self.columnscalefac = columnscalefac
        self.epoch = epoch
        self.batchsize = batchsize
        self.n_channels = n_channels
        self.scaleimgrow = np.int(imgrows/rowscalefac)
        self.scaleimgcol = np.int(imgcol/columnscalefac)
        self.n_channels=n_channels
        self.num_classes = num_classes
        self.shuffle = shuffle
        

datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
dataframe=df[:],
directory="../input/train/",
x_col="Id",
y_col="Target",
batch_size=32,

class_mode="categorical",
classes=["Nucleoplasm","Nuclear membrane","Nucleoli","Nucleoli fibrillar center","Nuclear speckles","Nuclear bodies","Endoplasmic reticulum","Golgi apparatus","Peroxisomes","Endosomes","Lysosomes","Intermediate filaments","Actin filaments","Focal adhesion sites","Microtubules","Microtubule ends","Cytokinetic bridge","Mitotic spindle","Microtubule organizing center","Centrosome","Lipid droplets","Plasma membrane","Cell junctions","Mitochondria","Aggresome","Cytosol","Cytoplasmic bodies","Rods & rings"],
target_size=(100,100))

classes=["Nucleoplasm","Nuclear membrane","Nucleoli","Nucleoli fibrillar center","Nuclear speckles","Nuclear bodies","Endoplasmic reticulum","Golgi apparatus","Peroxisomes","Endosomes","Lysosomes","Intermediate filaments","Actin filaments","Focal adhesion sites","Microtubules","Microtubule ends","Cytokinetic bridge","Mitotic spindle","Microtubule organizing center","Centrosome","Lipid droplets","Plasma membrane","Cell junctions","Mitochondria","Aggresome","Cytosol","Cytoplasmic bodies","Rods & rings"],


In [1]:
param = Mparam("../input/train/")

NameError: name 'Mparam' is not defined

In [ ]:
from skimage.transform import resize
class ImagePreprocessor:
    def __init__(self,mparam):
        self.parameter = mparam
        self.basepath = self.parameter.basepath
        self.scaleimgrow = self.parameter.scaleimgrow
        self.scaleimgcol = self.parameter.scaleimgcol
        self.n_channels = self.parameter.n_channels
    def preppro(self,image):
        image = self.resize(image)
        image = self.reshape(image)
        image = self.normalize(image)
        return image
    def resize(self,image):
        image = resize(image,(self.scaleimgrow,self.scaleimgcol))
        return image
    def reshape(self,image):
        image =np.reshape(image,(image.shape[0],image.shape[1],self.n_channels))
        return image
    def normalize(self,image):
        image /= 255
        return image
    def imgloader(self,imageid):
        image = np.zeros(shape=(512,512,1))
        image[:,:,0] = mpimg.imread(self.basepath+imageid+"_green"+".png") 
        return image[:,:,0]
        

In [ ]:
partition = {}
listt = []
listttest = []

In [ ]:
for id in df.Id.values:
    listt.append(id)

In [ ]:
partition['train'] = listt

In [ ]:
df2 = pd.read_csv("../input/sample_submission.csv")

In [ ]:
for id in df2.Id.values:
    listttest.append(id)

In [ ]:
partition['test'] = listttest

In [ ]:
labeldict={}


In [ ]:
for id,label in df.values:
    labeldict[id] = label 

In [ ]:
preprocessor = ImagePreprocessor(param)

In [ ]:
import keras

class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, list_IDs, labels, modelparameter, imagepreprocessor):
        self.current_epoch = 0
        self.params = modelparameter
        self.labels = labels
        self.list_IDs = list_IDs
        self.dim = (self.params.scaleimgrow, self.params.scaleimgcol)
        self.batch_size = self.params.batchsize
        self.n_channels = self.params.n_channels
        self.num_classes = self.params.num_classes
        self.shuffle = self.params.shuffle
        self.preprocessor = imagepreprocessor
        self.on_epoch_end()
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes, random_state=self.current_epoch)
            self.current_epoch += 1
    
    def get_targets_per_image(self, identifier):
        labellist = np.zeros(shape=(28))
        ls = df.loc[df['Id']==identifier].iloc[0,1]
        for i in range(0,28):
            if str(i) in ls:
                labellist[i] = 1
            else:
                labellist[i] = 0
        #ar = np.array([labellist.astype(int)])
        ar = np.array(labellist.astype(int))
        return ar
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.num_classes), dtype=int)
        # Generate data
        for i, identifier in enumerate(list_IDs_temp):
            # Store sample
            image = self.preprocessor.imgloader(identifier)
            image = self.preprocessor.preppro(image)
            X[i] = image
            # Store class
            y[i] = self.get_targets_per_image(identifier)
            
        return X, y
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

In [ ]:
training_generator = DataGenerator(partition['train'], df, param, preprocessor)


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(128,128,1)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(28, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit_generator(generator=training_generator,
                    epochs=1,
                    use_multiprocessing=True)

In [ ]:
submitdf = pd.read_csv("../input/sample_submission.csv")

the number of test files is *4 of submissions,so considering only green images and igoring the rest

In [ ]:
imgtest = mpimg.imread("../input/train/001838f8-bbca-11e8-b2bc-ac1f6b6435d0_green.png") 

In [ ]:
imgtest = preprocessor.preppro(imgtest)

In [ ]:
imgtest = imgtest.reshape((1, *imgtest.shape))

In [ ]:
res = model.predict(imgtest)

In [ ]:
res = res>0.5

In [ ]:
res

In [ ]:
#result storage initialisation
y = []


In [ ]:
#predictions
c=0
for id in submitdf.Id.values:
    imgtest = mpimg.imread("../input/test/"+id+"_green.png") 
    imgtest = preprocessor.preppro(imgtest)
    imgtest = imgtest.reshape((1, *imgtest.shape))
    res = model.predict(imgtest)
    #res = np.where(res>0.5,1,0)
    y.append(res)
    print(c)
    c=c+1
   
    

****

In [ ]:
y

In [ ]:
listtest = submitdf.Id



In [ ]:
test = pd.DataFrame(listtest)

In [ ]:
yt = np.array(y)

In [ ]:
print(yt.shape)
res = []

In [ ]:
for i in yt[:,0,:]:
    res.append(np.argmax(i))  


In [ ]:
res = np.array(res)
test2 = pd.DataFrame(res)

In [ ]:
finale = pd.concat([test,test2],axis=1)


In [ ]:
finale.columns = ["Id","Target"]

In [ ]:
finale.to_csv("submission.csv")